In [14]:
import logging
import ebooklib
from dotenv import load_dotenv
# from env_tokens import TELEGRAM_BOT_TOKEN, MISTRAL_API_ENDPOINT, PINECONE_API_KEY
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
from mistralai import Mistral
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
from ebooklib import epub
from bs4 import BeautifulSoup
from langchain.text_splitter import CharacterTextSplitter
import os

load_dotenv()

TELEGRAM_BOT_TOKEN = '7466857543:AAHWnxfKJ4hpS46zoI48CaRIYMP7aCir4BY'
MISTRAL_API_ENDPOINT = 'BbzDhJzmYPuASFVeZC7C6pLuHuC6qb6m'
PINECONE_API_KEY = 'pcsk_3jKr3Z_SrBc6DCo3JRmssPRaJ6yhRHkK4DK2VmdZyrmwSZu6ypEUuPYhFWyY743QiDadmZ'

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
logger = logging.getLogger(__name__)

# Токен Telegram бота
TELEGRAM_BOT_TOKEN = TELEGRAM_BOT_TOKEN

# API endpoint Mistral
MISTRAL_API_ENDPOINT = MISTRAL_API_ENDPOINT
MISTRAL_MODEL = 'mistral-medium'  # Replace with the name of your Mistral model
MISTRAL_CLIENT = Mistral(api_key=MISTRAL_API_ENDPOINT)

# API endpoint Pinecone
PINECONE_API_KEY = PINECONE_API_KEY
PINECONE_ENVIRONMENT = 'us-east-1'
INDEX_NAME = 'interview-qa2'

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Initialize SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#model = SentenceTransformer('intfloat/multilingual-e5-large')

# TODO Сделать БД?
ind_text_dict = {}


async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(
        'Hi! I am your Mistral RAG bot. Send me a message and I will retrieve and generate a response for you.')


async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_message = update.message.text
    logger.info(f"Received message: {user_message}")

    # Step 1: Retrieve relevant documents from the vector database
    retrieved_docs = retrieve_documents(user_message)

    # Step 2: Generate a response using Mistral API
    response = generate_response(user_message, retrieved_docs)

    await update.message.reply_text(response)


def retrieve_documents(query: str):
    # Convert the query to a vector
    query_vector = model.encode(query).tolist()

    # Query the Pinecone index with keyword arguments
    index = pc.Index(INDEX_NAME)
    results = index.query(vector=query_vector, top_k=5, include_metadata=True)
    
    # Log the retrieved results for debugging
    logger.info(f"Retrieved results: {results}")

    retrieved_docs = []
    for match in results['matches']:
        doc_id = match['id']
        try:
            retrieved_docs.append(ind_text_dict[doc_id])
        except KeyError:
            logger.warning(f"Document ID {doc_id} not found in local dictionary.")
    
    return retrieved_docs



def generate_response(query: str, documents: list):
    """Generate a response using Mistral API."""
    prompt = f"""
    You are an AI assistant designed to help users prepare for ML engineer interviews. 
    You have access to a knowledge base with information in English. 
    When a user asks a question, you should retrieve the relevant information from the knowledge base and then translate the response into the language of the user's question.
    Knowledge base: {documents}
    
    Here is the user's question:
    [{query}]
    
    Please provide the answer only in the language of the user's question.
        """

    # Send the prompt to Mistral API
    try:
        response = MISTRAL_CLIENT.chat.complete(
            model=MISTRAL_MODEL,
            messages=[{'role': "user", 'content': prompt}]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        logger.error(f"Error generating response: {e}")
        return "Sorry, I couldn't generate a response at the moment."


def extract_text_from_epub(file_path):
    book = epub.read_epub(file_path)
    text = []
    for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
        soup = BeautifulSoup(item.get_body_content(), 'html.parser')
        text.append(soup.get_text())
    return ' '.join(text)


def generate_embeddings(documents):
    embeddings = []
    for text_id, text in documents.items():
        embedding = model.encode(text)
        embeddings.append({'id': text_id, 'values': embedding.tolist()})
    return embeddings

2025-01-03 19:42:44,224 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-03 19:42:44,228 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-03 19:42:44,229 - pinecone_plugin_interface.logging - INFO - Installing plugin inference into Pinecone
2025-01-03 19:42:44,318 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-01-03 19:42:44,319 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2


In [15]:
INDEX_NAME = 'interview-qa2'
epub_folder = '/home/bullat/projects/rag/Interview-2.0/data'
documents = []
for filename in os.listdir(epub_folder):
    if filename.endswith('.epub'):
        file_path = os.path.join(epub_folder, filename)
        text = extract_text_from_epub(file_path)
        #TODO использовать name для ссылки на источник
        documents.append({'name': filename, 'text': text})

text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
texts = []
for doc in documents:
    chunks = text_splitter.split_text(doc['text'])
    for chunk in chunks:
        text_id = f"{doc['name']}_{chunks.index(chunk)}" 
        ind_text_dict[text_id] = chunk
    # texts.extend(text_splitter.split_text(doc['text']))
# print(ind_text_dict[0])

embeddings = generate_embeddings(ind_text_dict)
# print(embeddings[0])
# Create the index if it doesn't exist
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=len(embeddings[0]['values']),
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Upsert embeddings to the Pinecone index
index = pc.Index(INDEX_NAME)
index.upsert(embeddings)

/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):
2025-01-03 19:43:00,584 - langchain_text_splitters.base - WARNING - Created a chunk of size 3558, which is longer than the specified 2000
2025-01-03 19:43:00,585 - langchain_text_splitters.base - WARNING - Created a chunk of size 3734, which is longer than the specified 2000
2025-01-03 19:43:00,585 - langchain_text_splitters.base - WARNING - Created a 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-03 19:48:34,209 - urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(104, 'Connection reset by peer')': /indexes
2025-01-03 19:48:35,842 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-03 19:48:35,843 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference


{'upserted_count': 696}

In [16]:
import json

# Function to load questions from JSON file
def load_questions(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Function to save answers to JSON file
def save_answers(answers, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(answers, file, ensure_ascii=False, indent=4)

# Function to get answers from RAG
def get_answers(questions):
    answers = {}
    for q_id, question in questions.items():
        # Step 1: Retrieve relevant documents from the vector database
        retrieved_docs = retrieve_documents(question)

        # Step 2: Generate a response using Mistral API
        answer = generate_response(question, retrieved_docs)

        answers[q_id] = answer
    return answers

# Load questions from file
questions_file_path = '/home/bullat/projects/rag/Interview-2.0/question_for_test.json'
questions = load_questions(questions_file_path)

# Get answers from RAG
answers = get_answers(questions)

# Save answers to file
answers_file_path = '/home/bullat/projects/rag/Interview-2.0/answers.json'
save_answers(answers, answers_file_path)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-03 19:48:38,838 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-03 19:48:38,839 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-03 19:48:39,580 - __main__ - INFO - Retrieved results: {'matches': [{'id': 'John Mongan - Programming Interviews Exposed_ Coding Your '
                    'Way Through the Interview (2018, Wrox Press) - '
                    'libgen.li.epub_181',
              'score': 0.315634668,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds of '
                    'fully solved job interview questions from a wide range of '
                    'key topics in AI-Interviews AI (2020).epub_37',
              'score': 0.282967895,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds o

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-03 19:48:43,678 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-03 19:48:43,680 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-03 19:48:44,386 - __main__ - INFO - Retrieved results: {'matches': [{'id': 'John Mongan - Programming Interviews Exposed_ Coding Your '
                    'Way Through the Interview (2018, Wrox Press) - '
                    'libgen.li.epub_181',
              'score': 0.283690453,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds of '
                    'fully solved job interview questions from a wide range of '
                    'key topics in AI-Interviews AI (2020).epub_52',
              'score': 0.271557957,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds o

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

2025-01-03 20:13:27,957 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-03 20:13:27,958 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-03 20:13:28,592 - __main__ - INFO - Retrieved results: {'matches': [{'id': 'John Mongan - Programming Interviews Exposed_ Coding Your '
                    'Way Through the Interview (2018, Wrox Press) - '
                    'libgen.li.epub_181',
              'score': 0.326951474,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds of '
                    'fully solved job interview questions from a wide range of '
                    'key topics in AI-Interviews AI (2020).epub_37',
              'score': 0.279266417,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds o

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-03 20:13:40,020 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-03 20:13:40,021 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-03 20:13:40,646 - __main__ - INFO - Retrieved results: {'matches': [{'id': 'John Mongan - Programming Interviews Exposed_ Coding Your '
                    'Way Through the Interview (2018, Wrox Press) - '
                    'libgen.li.epub_181',
              'score': 0.296679169,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds of '
                    'fully solved job interview questions from a wide range of '
                    'key topics in AI-Interviews AI (2020).epub_37',
              'score': 0.265440583,
              'values': []},
             {'id': 'John Mongan - Programming Interviews Exposed_ Coding

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-03 20:13:46,479 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-03 20:13:46,480 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-03 20:13:47,103 - __main__ - INFO - Retrieved results: {'matches': [{'id': 'John Mongan - Programming Interviews Exposed_ Coding Your '
                    'Way Through the Interview (2018, Wrox Press) - '
                    'libgen.li.epub_181',
              'score': 0.291864961,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds of '
                    'fully solved job interview questions from a wide range of '
                    'key topics in AI-Interviews AI (2020).epub_37',
              'score': 0.280926913,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds o

In [17]:
import json
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load your data
with open('/home/bullat/projects/rag/Interview-2.0/answers.json', 'r', encoding='utf-8') as f:
    generated_answers = json.load(f)

with open('/home/bullat/projects/rag/Interview-2.0/Answer_fact.json', 'r', encoding='utf-8') as f:
    reference_answers = json.load(f)

# Initialize the SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Define function to calculate Cosine Similarity
def calculate_cosine_similarity(generated, reference):
    generated_vector = model.encode([generated])
    reference_vector = model.encode([reference])
    return cosine_similarity(generated_vector, reference_vector)[0][0]

# Define function to calculate BLEU Score
def calculate_bleu_score(generated, reference):
    reference_tokens = [reference.split()]
    generated_tokens = generated.split()
    return sentence_bleu(reference_tokens, generated_tokens)

# Calculate and print metrics
for key in generated_answers.keys():
    generated = generated_answers[key]
    reference = reference_answers[key]
    
    cosine_sim = calculate_cosine_similarity(generated, reference)
    bleu_score = calculate_bleu_score(generated, reference)
    
    print(f"Question ID: {key}")
    print(f"Generated Answer: {generated}")
    print(f"Reference Answer: {reference}")
    print(f"Cosine Similarity: {cosine_sim}")
    print(f"BLEU Score: {bleu_score}")
    print()



2025-01-03 20:13:51,560 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-01-03 20:13:51,561 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question ID: 0
Generated Answer: Алгоритм бинарного поиска — это эффективный алгоритм для поиска элемента в отсортированном массиве. Он основан на последовательном делении массива пополам до тех пор, пока не будет найден искомый элемент или массив станет пустым. Благодаря этому подходу, алгоритм бинарного поиска позволяет найти элемент за логарифмическое время, что значительно быстрее, чем линейный поиск. Алгоритм широко используется в различных областях, таких как информатика, математика и инженерия.
Reference Answer: Алгоритм бинарного поиска - это метод поиска элемента в отсортированном массиве, который делит массив пополам и проверяет центральный элемент, повторяя процесс с левой или правой частью массива до нахождения элемента.
Cosine Similarity: 0.9782422780990601
BLEU Score: 0.06190867264354498



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question ID: 98
Generated Answer: Ансамблевые методы в машинном обучении - это подход, который комбинирует несколько моделей машинного обучения для улучшения overall performance. Эти методы могут быть использоваться для улучшения точности, стабильности, и уменьшения overfitting. Существует несколько типов ансамблевых методов, таких как bagging, boosting, и stacking. Bagging создает несколько копий обучающей выборки и тренирует независимые модели на каждой копии. Затем результаты комбинируются путем голосования или среднего значения. Boosting создает последовательность моделей, где каждая модель улучшает ошибки предыдущей модели. Stacking комбинирует несколько моделей путем использования их предсказаний в качестве входных данных для другой модели. Эти методы часто используются в Kaggle соревнованиях и могут значительно улучшить overall performance моделей.
Reference Answer: Ансамблевые методы в машинном обучении объединяют несколько моделей для улучшения их точности и устойчивости. Осно

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question ID: 99
Generated Answer: To evaluate the quality of a model on test data, you can use various metrics such as accuracy, precision, recall, F1 score, area under the ROC curve (AUC-ROC), etc. These metrics help you measure the performance of your model and make informed decisions about its suitability for a particular task.

In Russian:

Для оценки качества модели на тестовых данных вы можете использовать различные метрики, такие как точность, полнота, точность, F1-мера, площадь под кривой ROC (AUC-ROC) и т.д. Эти метрики помогают вам измерить производительность модели и принять осмысленные решения о ее пригодности для конкретной задачи.
Reference Answer: Для оценки качества модели на тестовых данных я использую метрики, такие как точность (accuracy), полнота (recall), F-мера и AUC-ROC. Эти метрики позволяют оценить, насколько хорошо модель справляется с задачей классификации или регрессии на новых данных.
Cosine Similarity: 0.42467403411865234
BLEU Score: 0.07870263453430151



In [22]:
import json
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
import nltk

# Download required NLTK data
nltk.download('punkt')

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def simple_tokenize(text):
    # Simple whitespace tokenization with punctuation handling
    return [word.strip('.,!?()[]{}:;"\'') for word in text.split()]

def calculate_metrics(pred_answers, true_answers):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    smooth = SmoothingFunction()
    metrics = {}
    
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    bleu_scores = []
    
    for key in pred_answers:
        if key in true_answers:
            rouge = scorer.score(true_answers[key], pred_answers[key])
            for metric in rouge_scores:
                rouge_scores[metric].append(rouge[metric].fmeasure)
            
            reference = [simple_tokenize(true_answers[key].lower())]
            candidate = simple_tokenize(pred_answers[key].lower())
            
            if reference[0] and candidate:  # Check if tokens exist
                bleu = sentence_bleu(reference, candidate, 
                                   smoothing_function=smooth.method1)
                bleu_scores.append(bleu)
    
    # Calculate averages
    metrics['rouge1'] = np.mean(rouge_scores['rouge1']) if rouge_scores['rouge1'] else 0
    metrics['rouge2'] = np.mean(rouge_scores['rouge2']) if rouge_scores['rouge2'] else 0
    metrics['rougeL'] = np.mean(rouge_scores['rougeL']) if rouge_scores['rougeL'] else 0
    metrics['bleu'] = np.mean(bleu_scores) if bleu_scores else 0
    
    return metrics

# Load and evaluate
pred_answers = load_json('/home/bullat/projects/rag/Interview-2.0/answers.json')
true_answers = load_json('/home/bullat/projects/rag/Interview-2.0/Answer_fact.json')
metrics = calculate_metrics(pred_answers, true_answers)

print("\nEvaluation Metrics:")
print(f"ROUGE-1: {metrics['rouge1']:.4f}")
print(f"ROUGE-2: {metrics['rouge2']:.4f}")
print(f"ROUGE-L: {metrics['rougeL']:.4f}")
print(f"BLEU: {metrics['bleu']:.4f}")

[nltk_data] Downloading package punkt to /home/bullat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-01-03 20:24:03,151 - absl - INFO - Using default tokenizer.



Evaluation Metrics:
ROUGE-1: 0.1370
ROUGE-2: 0.0718
ROUGE-L: 0.1348
BLEU: 0.0329
